In [1]:
# %%
import os
from os import path
from pcdet.utils import common_utils
from pcdet.models import build_network, load_data_to_gpu
from pcdet.datasets import DatasetTemplate
from pcdet.config import cfg, cfg_from_yaml_file
import torch
import numpy as np
import argparse
import glob
from pathlib import Path

# from tools.visual_utils import visualize_utils as V
ABS_PATH_PREFIX = os.getcwd()


# %matplotlib inline
# %matplotlib notebook
%matplotlib widget
import matplotlib.pyplot as plt

In [2]:
#%%
class Args:
    def __init__(self):
        self.cfg_file = path.join(
            ABS_PATH_PREFIX,  'cfgs/kitti_models/second.yaml')
        self.data_path = path.join(
            ABS_PATH_PREFIX, '../data/kitti/')
        self.ckpt = path.join(ABS_PATH_PREFIX,
                              '../output/kitti_models/second/def/ckpt/checkpoint_epoch_50.pth')
        self.ext = '.bin'
        print("cfg _file  = ", self.cfg_file)


In [3]:

#%%

from pcdet.datasets.kitti.kitti_dataset import KittiDataset
os.chdir(ABS_PATH_PREFIX)
args = Args()

cfg_from_yaml_file(args.cfg_file, cfg)

# args, cfg = parse_config()

print("args = ", args)
print("cfg = ", cfg)

logger = common_utils.create_logger()
logger.info(
    '-----------------Quick Demo of OpenPCDet-------------------------')
demo_dataset = KittiDataset(
    dataset_cfg=cfg.DATA_CONFIG, class_names=cfg.CLASS_NAMES, training=True,
    root_path=Path(args.data_path), logger=logger
)
logger.info(f'Total number of samples: \t{len(demo_dataset)}')

model = build_network(model_cfg=cfg.MODEL, num_class=len(
    cfg.CLASS_NAMES), dataset=demo_dataset)
model.load_params_from_file(filename=args.ckpt, logger=logger, to_cpu=True)
model.cuda()
model.eval()
with torch.no_grad():
    for idx, data_dict in enumerate(demo_dataset):
        if idx == 1:
            break
        logger.info(f'Visualized sample index: \t{idx + 1}')
        data_dict = demo_dataset.collate_batch([data_dict])
        print("points fater collate = ", data_dict['points'][:,0])
        load_data_to_gpu(data_dict)
        pred_dicts, _ = model.forward(data_dict)

        # pred_dicts includes:
        #     pred_boxes, N x 7
        #     pred_scores, N
        #     pred_labels
        print(pred_dicts)

        print(data_dict.keys())

#         V.draw_scenes(
#             points=data_dict['points'][:,1:], ref_boxes=pred_dicts[0]['pred_boxes'],
#             ref_scores=pred_dicts[0]['pred_scores'], ref_labels=pred_dicts[0]['pred_labels']
#         )
#         mlab.show(stop=True)


logger.info('Demo done.')

2020-08-04 19:00:18,918   INFO  -----------------Quick Demo of OpenPCDet-------------------------
2020-08-04 19:00:19,081   INFO  Database filter by min points Car: 14357 => 13532
2020-08-04 19:00:19,082   INFO  Database filter by min points Pedestrian: 2207 => 2168
2020-08-04 19:00:19,083   INFO  Database filter by min points Cyclist: 734 => 705
2020-08-04 19:00:19,103   INFO  Database filter by difficulty Car: 13532 => 10759
2020-08-04 19:00:19,107   INFO  Database filter by difficulty Pedestrian: 2168 => 2075
2020-08-04 19:00:19,109   INFO  Database filter by difficulty Cyclist: 705 => 581


cfg _file  =  /home/gx/GitHub/OpenPCDet/tools/cfgs/kitti_models/second.yaml
args =  <__main__.Args object at 0x7f18618a9e50>
cfg =  {'ROOT_DIR': PosixPath('/home/gx/GitHub/OpenPCDet'), 'LOCAL_RANK': 0, 'CLASS_NAMES': ['Car', 'Pedestrian', 'Cyclist'], 'DATA_CONFIG': {'DATASET': 'KittiDataset', 'DATA_PATH': '../data/kitti', 'POINT_CLOUD_RANGE': [0, -40, -3, 70.4, 40, 1], 'DATA_SPLIT': {'train': 'train', 'test': 'val'}, 'INFO_PATH': {'train': ['kitti_infos_train.pkl'], 'test': ['kitti_infos_val.pkl']}, 'FOV_POINTS_ONLY': True, 'DATA_AUGMENTOR': {'DISABLE_AUG_LIST': ['placeholder'], 'AUG_CONFIG_LIST': [{'NAME': 'gt_sampling', 'USE_ROAD_PLANE': False, 'DB_INFO_PATH': ['kitti_dbinfos_train.pkl'], 'PREPARE': {'filter_by_min_points': ['Car:5', 'Pedestrian:5', 'Cyclist:5'], 'filter_by_difficulty': [-1]}, 'SAMPLE_GROUPS': ['Car:20', 'Pedestrian:15', 'Cyclist:15'], 'NUM_POINT_FEATURES': 4, 'DATABASE_WITH_FAKELIDAR': False, 'REMOVE_EXTRA_WIDTH': [0.0, 0.0, 0.0], 'LIMIT_WHOLE_SCENE': True}, {'NAME'

2020-08-04 19:00:19,129   INFO  Loading KITTI dataset
2020-08-04 19:00:19,250   INFO  Total samples for KITTI dataset: 3712
2020-08-04 19:00:19,251   INFO  Total number of samples: 	3712


self.exp =  [False, False, False]
feature_map_size = [array([64, 64]), array([64, 64]), array([64, 64])]


2020-08-04 19:00:22,178   INFO  ==> Loading parameters from checkpoint /home/gx/GitHub/OpenPCDet/tools/../output/kitti_models/second/def/ckpt/checkpoint_epoch_50.pth to CPU
2020-08-04 19:00:22,221   INFO  ==> Checkpoint trained from version: pcdet+0.2.0+b492d11
2020-08-04 19:00:22,361   INFO  ==> Done (loaded 163/163)
2020-08-04 19:00:22,416   INFO  Visualized sample index: 	1


points fater collate =  [0. 0. 0. ... 0. 0. 0.]


2020-08-04 19:00:22,796   INFO  Demo done.


[{'pred_boxes': tensor([[ 13.3165,  -5.5622,  -0.7407,   0.5050,   0.6311,   1.7220,   4.3537],
        [ 16.5955,  -0.7094,  -0.7130,   0.4931,   0.6480,   1.7849,   1.4360],
        [ 24.4222,  -8.3229,  -0.7994,   1.9388,   0.7536,   1.7211,   5.8068],
        [ 21.1515, -21.0523,  -0.7534,   0.5175,   0.5611,   1.6353,   2.2567],
        [ 38.1272, -10.7166,  -0.7235,   2.0843,   0.6283,   1.7151,   5.9935],
        [ 19.0421, -13.2067,  -0.7540,   0.4682,   0.4753,   1.6455,   1.0554],
        [ 12.1582,  -8.1080,  -0.8829,   0.5156,   0.6172,   1.6193,   3.7517],
        [ 13.3426, -14.7467,  -0.7520,   0.5611,   0.5365,   1.6390,   1.7077],
        [  9.0498, -14.5398,  -0.7732,   0.8596,   0.4614,   1.7120,   3.6718],
        [ 13.3157,  -4.6542,  -0.7238,   0.5105,   0.6317,   1.6957,   4.3880],
        [ 21.2894,  -2.0369,  -0.7792,   1.7948,   0.7333,   1.7474,   4.3251],
        [ 27.9227,  -5.8353,  -0.7691,   0.2512,   0.4454,   1.7682,   1.6847],
        [ 30.0848, -27.2

In [4]:
from mpl_toolkits.mplot3d import Axes3D
def visualize_pts(pts, fig=None, bgcolor=(1,1,1), fgcolor=(1.0, 1.0, 1.0),
                  show_intensity=True, size=(600, 600), draw_origin=True):
    if not isinstance(pts, np.ndarray):
        pts = pts.cpu().numpy()
    if fig is None:
        fig = plt.figure(figsize=(20, 10))
        
        ax = fig.add_subplot(111, projection='3d')
        ax.set_facecolor(bgcolor)

    if show_intensity:

        ax.scatter(pts[:, 0], pts[:, 1], pts[:, 2], c=pts[:,3], s=0.01, marker=',')
    else:
        ax.scatter(pts[:, 0], pts[:, 1], pts[:, 2], s=0.01, marker=',')
        
    if draw_origin:
        ax.plot([0,3], [0,0], [0,0], color=(0,0,1))
        ax.plot([0,0], [0,3], [0,0], color=(0,1,0))
        ax.plot([0,0], [0,0], [0,3], color=(1,0,0))
    
    return ax

    # if draw_origin:

In [5]:
fig = plt.figure()
points = data_dict['points']

plt.plot(points[:,2])
fig = plt.figure()
plt.plot(points[:,1])
fig = plt.figure()
plt.plot(points[:,0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
ax = visualize_pts(points)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
def draw_grid(x1, y1, x2, y2, ax, color=(1,0.5,0.5)):
    ax.plot([x1, x1], [y1, y2], [0, 0], color=color, linewidth=1)
    ax.plot([x2, x2], [y1, y2], [0, 0], color=color, linewidth=1)
    ax.plot([x1, x2], [y1, y1], [0, 0], color=color, linewidth=1)
    ax.plot([x1, x2], [y2, y2], [0, 0], color=color, linewidth=1)
    return ax

def draw_multi_grid_range(ax, grid_size=20, bv_range=(-60, -60, 60, 60)):
    for x in range(bv_range[0], bv_range[2], grid_size):
        for y in range(bv_range[1], bv_range[3], grid_size):
            ax = draw_grid(x, y, x + grid_size, y+grid_size, ax) 
    return ax

In [8]:
draw_multi_grid_range(ax, bv_range=(0, -40, 80, 40))

<Axes3DSubplot:>

In [9]:
def check_numpy_to_torch(x):
    if isinstance(x, np.ndarray):
        return torch.from_numpy(x).float(), True
    return x, False


def rotate_points_along_z(points, angle):
    """
    Args:
        points: (B, N, 3 + C)
        angle: (B), angle along z-axis, angle increases x ==> y
    Returns:

    """
    points, is_numpy = check_numpy_to_torch(points)
    angle, _ = check_numpy_to_torch(angle)

    cosa = torch.cos(angle)
    sina = torch.sin(angle)
    zeros = angle.new_zeros(points.shape[0])
    ones = angle.new_ones(points.shape[0])
    rot_matrix = torch.stack((
        cosa,  sina, zeros,
        -sina, cosa, zeros,
        zeros, zeros, ones
    ), dim=1).view(-1, 3, 3).float()
    points_rot = torch.matmul(points[:, :, 0:3], rot_matrix)
    points_rot = torch.cat((points_rot, points[:, :, 3:]), dim=-1)
    return points_rot.numpy() if is_numpy else points_rot


def boxes_to_corners_3d(boxes3d):
    """
        7 -------- 4
       /|         /|
      6 -------- 5 .
      | |        | |
      . 3 -------- 0
      |/         |/
      2 -------- 1
    Args:
        boxes3d:  (N, 7) [x, y, z, dx, dy, dz, heading], (x, y, z) is the box center

    Returns:
    """
    boxes3d, is_numpy = check_numpy_to_torch(boxes3d)

    template = boxes3d.new_tensor((
        [1, 1, -1], [1, -1, -1], [-1, -1, -1], [-1, 1, -1],
        [1, 1, 1], [1, -1, 1], [-1, -1, 1], [-1, 1, 1],
    )) / 2

    corners3d = boxes3d[:, None, 3:6].repeat(1, 8, 1) * template[None, :, :]
    corners3d = rotate_points_along_z(corners3d.view(-1, 8, 3), boxes3d[:, 6]).view(-1, 8, 3)
    corners3d += boxes3d[:, None, 0:3]

    return corners3d.numpy() if is_numpy else corners3d

In [10]:

def draw_corners3d(corners3d, ax, color=(1, 1, 1), line_width=1, cls=None, tag='', max_num=500, tube_radius=None):
    """
    :param corners3d: (N, 8, 3)
    :param fig:
    :param color:
    :param line_width:
    :param cls:
    :param tag:
    :param max_num:
    :return:
    """
    num = min(max_num, len(corners3d))
    for n in range(num):
        b = corners3d[n]  # (8, 3)

        if cls is not None:
            if isinstance(cls, np.ndarray):
                ax.text(b[6, 0], b[6, 1], b[6, 2], '%.2f' % cls[n], fontsize=0.3, color=color)
            else:
                ax.text(b[6, 0], b[6, 1], b[6, 2], '%s' % cls[n], fontsize=0.3, color=color)

        for k in range(0, 4):
            i, j = k, (k + 1) % 4
            ax.plot([b[i, 0], b[j, 0]], [b[i, 1], b[j, 1]], [b[i, 2], b[j, 2]], color=color,
                        linewidth=line_width)

            i, j = k + 4, (k + 1) % 4 + 4
            ax.plot([b[i, 0], b[j, 0]], [b[i, 1], b[j, 1]], [b[i, 2], b[j, 2]], color=color,
                        linewidth=line_width)

            i, j = k, k + 4
            ax.plot([b[i, 0], b[j, 0]], [b[i, 1], b[j, 1]], [b[i, 2], b[j, 2]], color=color,
                        linewidth=line_width)

        i, j = 0, 5
        ax.plot([b[i, 0], b[j, 0]], [b[i, 1], b[j, 1]], [b[i, 2], b[j, 2]], color=color,
                    linewidth=line_width)
        i, j = 1, 4
        ax.plot([b[i, 0], b[j, 0]], [b[i, 1], b[j, 1]], [b[i, 2], b[j, 2]], color=color,
                    linewidth=line_width)

    return ax



In [11]:
ref_boxes=pred_dicts[0]['pred_boxes'].cpu().numpy()
ref_scores=pred_dicts[0]['pred_scores'].cpu().numpy()
ref_labels=pred_dicts[0]['pred_labels'].cpu().numpy()

ref_corners3d = boxes_to_corners_3d(ref_boxes)
ax = draw_corners3d(ref_corners3d, ax, color=(0,1,0), cls=ref_scores, max_num=100)

box_colormap = [
    [1, 1, 1],
    [0, 1, 0],
    [0, 1, 1],
    [1, 1, 0],
]

gt_boxes = data_dict['gt_boxes']
if gt_boxes is not None:
    corners3d = boxes_to_corners_3d(gt_boxes)
    ax = draw_corners3d(corners3d, ax, color=(0, 0, 1), max_num=100)

    if ref_boxes is not None:
        ref_corners3d = boxes_to_corners_3d(ref_boxes)
        if ref_labels is None:
            ax = draw_corners3d(ref_corners3d, ax, color=(0, 1, 0), cls=ref_scores, max_num=100)
        else:
            for k in range(ref_labels.min(), ref_labels.max() + 1):
                cur_color = tuple(box_colormap[k % len(box_colormap)])
                mask = (ref_labels == k)
                ax = draw_corners3d(ref_corners3d[mask], ax, color=cur_color, cls=ref_scores[mask], max_num=100)

In [12]:
data_dict.keys()

dict_keys(['points', 'frame_id', 'gt_boxes', 'use_lead_xyz', 'voxels', 'voxel_coords', 'voxel_num_points', 'image_shape'])